##### Imports & Constants

In [ ]:
import pathlib
import pandas as pd

import sklearn.model_selection as ms
import sklearn.ensemble as ske
import sklearn.metrics as skm

FINAL_DATA_PATH = pathlib.Path('..') / 'data' / 'final' / 'final_data.csv'
PCA_DATA_PATH = pathlib.Path('..') / 'data' / 'final' / 'pca_data.csv'

final_data = pd.read_csv(FINAL_DATA_PATH)
pca_data = pd.read_csv(PCA_DATA_PATH)

X, y = final_data.drop('position', axis=1), final_data['position']
pca_X, pca_y = pca_data, final_data['position']

# Split data into train and test sets
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
pca_X_train, pca_X_test, pca_y_train, pca_y_test = ms.train_test_split(pca_X, pca_y, test_size=0.2, random_state=42, shuffle=True)

##### Inital Model

In [ ]:
rf = ske.RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# Metrics
print(skm.accuracy_score(y_test, y_pred))
print(skm.f1_score(y_test, y_pred, average='weighted'))


##### Inital Model (PCA)

In [ ]:
rf = ske.RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(pca_X_train, pca_y_train)
y_pred = rf.predict(pca_X_test)

# Metrics
print(skm.accuracy_score(pca_y_test, y_pred))
print(skm.f1_score(pca_y_test, y_pred, average='weighted'))
